# Notebook to showcase the basic functioning of the wfc3_dash module

Table of Contents: <br>
1. <a href='#imports'>Imports</a>
2. <a href='#introduction'>Introduction</a>
3. <a href='#downloads'>Downloading Relevant Data </a>
4. <a href='#DASH'>Running DASH </a> <br>
    a) <a href='#object'> Creating DashData object <br> </a>
    b) <a href='#diff_files'> Create diff files <br> </a>
    c) <a href='#asn_table'> Create association tables <br> </a>
    e) <a href='#subtract_ext'> Subtract background from new FLT's <br> </a>
    f) <a href='#cosmic_rays'> Fix cosmic rays <br> </a>
    g) <a href='#align_each_other'> Align reads to each other <br> </a>
    h) <a href='#align_cat'> Align reads to catalog <br> </a>
    i) <a href='#align_both'> Align reads to each other, then to gaia catalog <br> </a>
5. <a href='#mosaic'> Creating Mosaic </a>

<a id='imports'></a>

## Imports

* *astroquery.mast Observations* used to download IMA files from the MAST HST archive
* *astropy.io import fits* used to open the files
* *matplotlib.pyplot* used to plot the images
* *numpy* used for some math

In [ ]:
from astroquery.mast import Observations
from astropy.io import fits 
from astropy.table import Table
import matplotlib.pyplot as plt 
import numpy as np

from astropy.io import ascii

from glob import glob
from drizzlepac import astrodrizzle

import os

%matplotlib notebook 

<a id='introduction'></a>

## Introduction

The wfc3_dash submodule of wfc3_tools is used to reduce the effects of the spacecraft drift for WFC3/IR images taken in DASH mode (i.e. under GYRO control, rather than under Fine-Guide-Sensor control)

This notebook works on a single .flt file but can be easily adapted to work on all exposures within a DASH visit or even a DASH program

<a id='downloads'></a>

## Downloading some relevant data

#### Get the table of observations associated to GO-14114 (PI van Dokkum, the first proposal to use the DASH mode)

In [ ]:
obsTable = Observations.query_criteria(proposal_id=['14114'])

#### Get the full list of products associated to the table and restrict the list to IMA files

In [ ]:
product_list = Observations.get_product_list(obsTable)
BM = (product_list['productSubGroupDescription']  == 'IMA') 
product_list = product_list[BM]

#### Display (part of) the IMA files list

In [ ]:
product_list.show_in_notebook(display_length=5)

#### Pick a single exposure file to work on

In [ ]:
myID = product_list['obsID'][0:1]

#### Download the IMA and FLT files for that exposure. The standard pipeline-FLT will be used for comparison with the detrended final product

In [ ]:
download = Observations.download_products(myID,mrp_only=False,productSubGroupDescription=['IMA','FLT'])

#### Display the results of the download operation

In [ ]:
download

#### Read the files that were just downloaded locally 

In [ ]:
#have path be everything minus last 8 characters (ima.fits)
localpathtofile = download['Local Path'][0][:-8]
localpathtofile

original_ima = fits.open(localpathtofile+'ima.fits')
original_flt = fits.open(localpathtofile+'flt.fits')
original_ima.info()

#### Plot the individual reads of the IMA file
Note: the individual 'SCI' extensions are stored in reverse order, with 'SCI', 1 corresponding to the last read

In [ ]:
nsamp = original_ima[0].header['NSAMP']
print('NSAMP',nsamp)
fig,axarr = plt.subplots((nsamp+3)//4,4, figsize=(9,3*((nsamp+3)//4)))

for i in range(1,4*((nsamp+3)//4)+1):

    row = (i-1)//4
    col = (i-1)%4
    if (i <= nsamp):
        immed = np.nanmedian(original_ima['SCI',i].data)
        stdev = np.nanstd(original_ima['SCI',i].data)
        axarr[row,col].imshow(original_ima['SCI',i].data,clim=[immed-.3*stdev,immed+.5*stdev],cmap='Greys',origin='lower')
        axarr[row,col].set_title('SCI '+str(i))
        axarr[row,col].set_xticks([]) 
        axarr[row,col].set_yticks([]) 
    else:
        fig.delaxes(axarr[row,col])

fig.tight_layout()

<a id='DASH'></a>

## Run the individual steps of the DASH pipeline

Run the DASH pipeline for a single exposure.  
This procedure showcases the capabilities and customization options of the DASH pipeline.

#### This cell is inserted temporarily to allow for relative imports until the whole wfc3_dash submodule is properly packaged and installed within the wfc3_tools module

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from reduce_dash import DashData

<a id='object'></a>

### 1. Create a DashData object using the path to the ima file we have downloaded above

In [ ]:
myDash = DashData(localpathtofile+'ima.fits', flt_file_name=localpathtofile+'flt.fits')

<a id='diff_files'></a>

### 2. Create diff files

A diff file contains the counts accumulated between two reads.  
The diff files are written to disk in a directory named ./diff under the current working directory (cwd).  
In creating diff files, the first difference, between the 1-st and 0-th read is ignored becuase of   
its very short expsoure time of 2.9 seconds, resulting in a noisy image.

In order to create a correct error extension, the split_ima() method calls the utils.get_flat() function and the utils.get_IDCtable function.  
The get_flat function reads the name of the flat field used for calibrating the ima images from the ima file header.  
The get_IDCtable reads the name of image distortion correction table, a reference file containing distortion coefficients that are used to correct for distortion in MAST drizzled data products.  
If the flat file is not present locally in a directory named ./iref under the cwd, get_flat() will download   
the flat field file from the CRDS database https://hst-crds.stsci.edu/unchecked_get/references/hst/ 
and place it in ./iref . Similarly for the IDC table.

In [ ]:
myDash.split_ima()

#### Plot the diff files

In [ ]:
ndiff = len(myDash.diff_files_list)
print('Number of diff files',ndiff)
fig,axarr = plt.subplots((ndiff+3)//4,4, figsize=(9,3*((ndiff+3)//4)))

for i in range(4*((ndiff+3)//4)):

    row = (i)//4
    col = (i)%4
    if (i < ndiff):
        diff_i = fits.open(myDash.diff_files_list[i]+'_diff.fits')
        immed = np.nanmedian(diff_i['SCI'].data)
        stdev = np.nanstd(diff_i['SCI'].data)
        axarr[row,col].imshow(diff_i['SCI'].data,clim=[immed-.3*stdev,immed+.5*stdev],cmap='Greys',origin='lower')
        axarr[row,col].set_title('Diff:'+str(i+1))
        axarr[row,col].set_xticks([]) 
        axarr[row,col].set_yticks([]) 
    else:
        fig.delaxes(axarr[row,col])

fig.tight_layout()

<a id='asn_table'></a>

### 3. Create an association file

This file mimics a typical association file for dithered exposures, that is used by astrodrizzle   
to align and stack multiple exposures taken at the same sky position with small dithers.  
We exploit the fact that a WFC3/IR exposure taken under gyro control can be effectively split into   
individual pseudo-exposures (the diff images).  
Astrodrizzle can treat such pseudo-expsoures as individual dithers, and combine them.

In [ ]:
myDash.make_pointing_asn()

#### Show the content of the asn file

In [ ]:
asn_filename = 'diff/{}_asn.fits'.format(myDash.root)
asn_table = Table(fits.getdata(asn_filename, ext=1))
asn_table.show_in_notebook()

### 4. Create Segmentation Map

#### Create segmentation map from original FLT

Make segmentation map from original FLT image to assist with background subtraction and fixing of cosmic ray flags.  

In [ ]:
myDash.create_seg_map()

View segmentation map.

In [ ]:
segmap = fits.getdata('icxe01tyq_seg.fits')
fig = plt.figure(figsize=(10, 12.5))
plt.imshow(segmap, origin='lower', vmin=0.6, vmax=0.7, cmap='Greys_r')

Print and read source list.

In [ ]:
sourcelist = ascii.read('icxe01tyq_source_list.dat')
print(sourcelist)

#### Create segmentation map and source list from diff files

Make source lists from difference files so that TweakReg can better align to catalogs

Make list of difference files that contain the full path name.

In [ ]:
diffpath = os.path.dirname(os.path.abspath('diff/{}_*_diff.fits'.format('icxe01tyq')))
cat_images=sorted([os.path.basename(x) for x in glob('diff/{}_*_diff.fits'.format('icxe01tyq'))])

sc_diff_files = [diffpath + '/' + s for s in cat_images]

In [ ]:
myDash.diff_seg_map(cat_images=sc_diff_files)

<a id='subtract_ext'></a>

### 5. Subtract Background from diff files

Subtract background from the individual reads taken from the original IMA file using the DRZ and SEG imaged produced in the background subtraction of the original FLT.  
By default, this function will subtract the background and write it to the header. Setting parameter subtract to False will not subtract the background and only write it to the header.  
Set parameter reset_stars_dq to True to reset cosmic rays within objects to 0 (because the centers of the stars are flagged).

In [ ]:
myDash.subtract_background_reads()

<a id='cosmic_rays'></a>

### 6. Fix Cosmic Rays

In [ ]:
myDash.fix_cosmic_rays()

<a id='align_each_other'></a>

### 7a. Align reads to each other

Align reads to one another by aligning each to the first diff file.  
Uses TweakReg to update the WCS information in the headers of the diff files, then drizzles the images together using Astrodrizzle.  
Refer to documentation to customize parameters for TweakReg and AstroDrizzle. 

NOTE: UnboundLocalError: local variable 'sig' referenced before assignment <br>
--> Can be solved by lowering threshold parameter

In [ ]:
myDash.align(updatehdr=False, updateWCS=False, astrodriz=False)

Print the shifts file to analyze how well the alignment went.  
Do not update header until shifts are satisfactory. 

In [ ]:
print(open('shifts_icxe01tyq.txt').read())

Update header and WCS information, then plot final drizzled image.

In [ ]:
myDash.align(threshold=20.)

In [ ]:
sci = fits.getdata('icxe01tyq_drz_sci.fits')
og_flt = fits.getdata('mastDownload/HST/icxe01tyq/icxe01tyq_flt.fits')

fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(1,2,2)
ax2 = fig.add_subplot(1,2,1)

ax1.set_title('DASH Pipeline Reduced Science File')
ax2.set_title('Original IMA (not reduced using pipeline)')

ax1.imshow(sci, vmin=-0.05, vmax=0.4, cmap='Greys', origin='lower', aspect="auto")
ax2.imshow(og_flt, vmin=0.6, vmax=1.15, cmap='Greys', origin='lower', aspect="auto")

<a id='align_cat'></a>

### 7b. Align reads to COSMOS<br>

#### Align images. <br>
Parameters ref_catalog and ref_image denote the reference catalog and reference image, respectively.  
Note: Must be connected to STScI network in order to align to Gaia (specifically to use updatewcs function on the input images for TweakReg). To not use this function, set parameter updatewcsfn to False.

If an error arises, try lowering the threshold.

In [ ]:
from utils import get_IDCtable

In [ ]:
get_IDCtable('mastDownload/HST/icxe01tyq/icxe01tyq_ima.fits')

In [ ]:
import urllib.request
import os

if not os.path.exists('ref_cat'):
    os.mkdir('ref_cat')

print('Beginning catalog download...')

url = 'https://irsa.ipac.caltech.edu/data/COSMOS/tables/photometry/cosmos_acs_iphot_200709.tbl'
urllib.request.urlretrieve(url, 'ref_cat/refcat.tbl')

Navigate to the appropriate directory, then type the following command in your terminal in order to create reference table consisting of only the RA and DEC values: <br>

awk '{ print \\$35, \\$36 }' ref_cat/refcat.tbl > ref_cat/cosmos_ra_dec.tbl

Remove header of catalog.

In [ ]:
n = 124
nfirstlines = []

with open('ref_cat/cosmos_ra_dec.tbl') as f, open("ref_cat/temp_cat.dat", "w") as out:
    for x in range(n):
        nfirstlines.append(next(f))
    for line in f:
        out.write(line)

os.remove('ref_cat/cosmos_ra_dec.tbl')
os.rename("ref_cat/temp_cat.dat", 'ref_cat/cosmos_ra_dec.tbl')

Align to COSMOS catalog.

In [ ]:
myDash.align(align_method='CATALOG', 
             ref_catalog='ref_cat/cosmos_ra_dec.tbl', 
             cat_file='diff_catfile.cat',
             threshold=40., searchrad=60., 
             cw=3.5, 
             updatehdr=False, 
             updateWCS=False, 
             astrodriz=False)

Inspect the shifts file to see if method used produced sufficient results. Notice that some diff files could not sufficiently line up with Gaia. It is up to the user to determine whether this is sufficient or not.

In [ ]:
print(open('shifts_icxe01tyq.txt').read())

Update header and plot aligned science images.

In [ ]:
myDash.align(align_method='CATALOG', 
             ref_catalog='ref_cat/cosmos_ra_dec.tbl', 
             cat_file='diff_catfile.cat',
             threshold=40., searchrad=60., cw=3.5, wcsname='DASH2')

In [ ]:
sci = fits.getdata('icxe01tyq_drz_sci.fits')

fig = plt.figure(figsize=(10, 10))

plt.imshow(sci, vmin=-0.05, vmax=0.4, cmap='Greys', origin='lower')

<a id='align_both'></a>

### 7c. Align reads to each other, then align final read to COSMOS

Will most likely give better results, especially if not enough sources are found using catalog method alone.

Align sources to each other using TweakReg.

In [ ]:
myDash.align(astrodriz=False)

Align updated images to COSMOS.

In [ ]:
myDash.align(align_method='CATALOG', 
             ref_catalog='ref_cat/cosmos_ra_dec.tbl', 
             cat_file='diff_catfile.cat',
             threshold=40., searchrad=60., cw=3.5, wcsname='DASH3')

Inspect the shifts file to see if method used produced sufficient results. Then plot final drizzled image.

In [ ]:
print(open('shifts_icxe01tyq.txt').read())

In [ ]:
sci = fits.getdata('icxe01tyq_drz_sci.fits')

fig = plt.figure(figsize=(20, 20))

plt.imshow(sci, vmin=-0.05, vmax=0.4, cmap='Greys', origin='lower')

<a id='mosaic'></a>

## Create mosaic

Creates mosaic from multiple exposures using DashData class.

Download first 6 exposures.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from reduce_dash import main

In [ ]:
obsTable = Observations.query_criteria(proposal_id=['14114'])

product_list = Observations.get_product_list(obsTable)
BM = (product_list['productSubGroupDescription']  == 'IMA') 
product_list = product_list[BM]

myID = product_list['obsID'][0:6]

download = Observations.download_products(myID,mrp_only=False,productSubGroupDescription=['IMA','FLT'])

Run main function, aka the DASH pipeline, on downloaded files.

In [ ]:
ima_exposures = glob('./mastDownload/HST/*/*_ima.fits')
flt_exposures = glob('./mastDownload/HST/*/*_flt.fits')

For simplicity, we will align the reads the each other.

In [ ]:
for ima_exp, flt_exp in zip(ima_exposures, flt_exposures):
    main(ima_exp, flt_exp, astrodriz=False, subtract_background=False, wcsname='DASH4')

Drizzle the updated difference images together from all 6 exposures.

In [ ]:
diff_files = 'diff/icxe01*_*_diff.fits'

astrodrizzle.AstroDrizzle(diff_files, output='mosaic')

Plot the final mosaic.

In [ ]:
sci = fits.getdata('mosaic_drz_sci.fits')

fig = plt.figure(figsize=(10, 10))

plt.imshow(sci, vmin=-0.05, vmax=0.4, cmap='greys', origin='lower')